<a href="https://colab.research.google.com/github/MILLERMARRU/rec_indhtface/blob/main/Reconocimiento_insgface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit insightface onnxruntime opencv-python-headless pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 382.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00


In [2]:
!ngrok config add-authtoken 2rPOA88C1NvlhE7pXulxY6VlUMA_4LMGUuiywutna8pigaZ4A

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [15]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
from PIL import Image
from insightface.app import FaceAnalysis

# Cargar modelo (una sola vez)
@st.cache_resource
def load_model():
    app = FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
    app.prepare(ctx_id=0)
    return app

app = load_model()

st.set_page_config(layout="wide")
st.title("🧠 Detector Facial con InsightFace")

col1, col2 = st.columns(2)

with col1:
    st.markdown("### 📸 Toma una foto o carga una imagen")

    camera_image = st.camera_input("Tomar foto")
    uploaded_file = st.file_uploader("Subir imagen", type=["jpg", "jpeg", "png"])

with col2:
    if camera_image or uploaded_file:
        with st.spinner("🔍 Detectando rostros..."):
            if camera_image:
                image = Image.open(camera_image).convert("RGB")
            else:
                image = Image.open(uploaded_file).convert("RGB")

            img_np = np.array(image)
            img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

            faces = app.get(img_bgr)

            for face in faces:
                x1, y1, x2, y2 = map(int, face.bbox)
                cv2.rectangle(img_np, (x1, y1), (x2, y2), (0, 255, 0), 2)

        st.success(f"✅ Rostros detectados: {len(faces)}")
        st.image(img_np, caption="Resultado", channels="RGB", use_container_width=True)
    else:
        st.info("📷 Toma una foto o carga una imagen para comenzar")



Overwriting app.py


In [16]:
from pyngrok import ngrok
import os

# Cerrar cualquier túnel anterior
ngrok.kill()

# Iniciar Streamlit
port = 8501
public_url = ngrok.connect(port)
print("🌐 URL pública:", public_url)

# Ejecutar la app (modo background)
!streamlit run app.py &> /dev/null &


🌐 URL pública: NgrokTunnel: "https://86eec49e9c60.ngrok-free.app" -> "http://localhost:8501"
